# 1. Парсинг всех пленарних засідань ВР

Установка необходимых пакетов

In [15]:
!pip install Scrapy==1.6.0
!pip install Twisted==19.2.0

  Using cached Twisted-19.2.0.tar.bz2 (3.1 MB)
  ERROR: Command errored out with exit status 1:
   command: /home/vadym/Projects/temp/texty_vr/venv/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-q98flhkd/Twisted/setup.py'"'"'; __file__='"'"'/tmp/pip-install-q98flhkd/Twisted/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-t38px8n8
       cwd: /tmp/pip-install-q98flhkd/Twisted/
  Complete output (6 lines):
  usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: setup.py --help [cmd1 cmd2 ...]
     or: setup.py --help-commands
     or: setup.py cmd --help
  
  error: invalid command 'bdist_wheel'
  ----------------------------------------
  ERROR: Failed building wheel for Twisted
  Running setup.py clean for Twisted
Failed to build Twisted
  Attempting uninstall: Twis

In [1]:
import scrapy
import datetime
import json
from asyncio import Queue
from multiprocessing import Process
from scrapy import crawler
from twisted.internet import reactor

from scrapy.crawler import CrawlerProcess

Парсить результаты будем из **https://www.rada.gov.ua/news/hpz9/page/N/**, где **N** -- номер страницы -- на текущий момент от 1 до 308

Крайне примитивный парсер дает дату и название события, его id во внутренней системе и ссылку на результаты голосования

In [2]:
def get_all_links():
    answer = []

    for i in range(1, 309):
        answer.append(f"https://www.rada.gov.ua/news/hpz9/page/{i}/")

    return answer

In [7]:
class VrSessionSpider(scrapy.Spider):
    name = "session"
    start_urls = get_all_links()

    def parse(self, response):
        timeline_block = response.css("div.timeline-block")
        for block in timeline_block:
            date = "".join(block.css('div.timeline-block__date')[0].css('::text').extract()).strip().replace(" ", " ")
            time = "".join(block.css('span.time')[0].css('::text').extract()).strip()

            
            for a in block.css('a'):
                answer = {
                    "date": date,
                    "time": time,
                    "text": "".join(a.css('::text').extract()).strip(),
                    "link": a.attrib["href"],
                    "id": int(a.attrib["href"].split("=")[1])
                }

                yield answer

In [4]:
# by default, CrawlerProcess is not restartable, so here it is a wrapper to make it run more times

def run_spider(spider, json_file):
    def f(q):
        try:
            runner = crawler.CrawlerRunner({
                'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                'FEED_FORMAT': 'json',
                'FEED_URI': json_file
            })
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        return result

In [8]:
JSON_FILE = 'vr_session_parse_' + str(datetime.datetime.now()) + '.json'
run_spider(VrSessionSpider, JSON_FILE)

<ipython-input-4-7d3601ce61a4>:14: RuntimeWarning: coroutine 'Queue.put' was never awaited
  q.put(None)


<coroutine object Queue.get at 0x7fd0fac47840>

In [9]:
json_file = open(JSON_FILE, "r+")

text = json_file.read()
json_file.seek(0)
json_info = json.loads(text)
new_text = json.dumps(json_info, ensure_ascii=False, indent=1)

json_file.write(new_text)
json_file.truncate()
json_file.close()

В полученном файле список всех рассматриваемых текущим созывом вопросов - тема, дата и время и ссылка на страницу с результатами голосования -- именно по этому списку будут парситься результаты поименного голосования

Что еще можно сделать с этим файлом
* Прогнать названия заголосков вопросов через цепь Маркова и получить модельку генерации абсурда вроде вот этой (https://t.me/regionality/918) или (https://t.me/NeuralShit/1314)
* Посмотреть на время принятия решения -- как часто и что именно принимают депутаты в непривычные для них часы работы (как минимум, однажды заседали до двух ночи) и что при этом делают и как именно голосуют (как правило, рассматривают спам из поправок)